<a href="https://colab.research.google.com/github/molabokchi/bokchi_open_lab/blob/main/torch_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -qq

# Fetch audio, video and other data files to log
!git clone https://github.com/wandb/examples.git
!pip install soundfile -qq

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 6.7 MB/s eta 0:00:00
Cloning into 'examples'...
remote: Enumerating objects: 6689, done.
remote: Counting objects: 100% (287/287), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 6689 (delta 149), reused 250 (delta 126), pack-reused 6402
Receiving objects: 100% (6689/6689), 165.73 MiB | 17.73 MiB/s, done.
Resolving deltas: 100% (3658/3658), done.


In [2]:
import wandb

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # noqa: N812
import torch.optim as optim

In [5]:
# experiment init & config hyperparam
wandb.init(project= "torch_test", 
                config={ 'learning_rate':0.001,'context_size':2 , 'embedding_dim':10})

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
#test example
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [
    ([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
    for i in range(len(test_sentence) - 2)
]
_hook_handles ={}



vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

has_cuda = torch.cuda.is_available()

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), wandb.config.embedding_dim, wandb.config.context_size)
model = model.cuda() if has_cuda else model
optimizer = optim.SGD(model.parameters(), lr=wandb.config.learning_rate)
wandb.watch(model, log_freq=1000)

for i in range(100):
    print("epoch :" + str(i))
    total_loss = 0
    for batch_i, (context, target) in enumerate(trigrams):

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor(
            [word_to_ix[w] for w in context], dtype=torch.long
        )
        context_idxs = context_idxs.cuda() if has_cuda else context_idxs

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        target = torch.tensor([word_to_ix[target]], dtype=torch.long)
        target = target.cuda() if has_cuda else target
        loss = loss_function(log_probs, target)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
        if batch_i%10 ==0:
          wandb.log({"batch_loss": loss.item()})
    losses.append(total_loss)
print(losses)  # The loss decreased ev

epoch :0
epoch :1
epoch :2
epoch :3
epoch :4
epoch :5
epoch :6
epoch :7
epoch :8
epoch :9
epoch :10
epoch :11
epoch :12
epoch :13
epoch :14
epoch :15
epoch :16
epoch :17
epoch :18
epoch :19
epoch :20
epoch :21
epoch :22
epoch :23
epoch :24
epoch :25
epoch :26
epoch :27
epoch :28
epoch :29
epoch :30
epoch :31
epoch :32
epoch :33
epoch :34
epoch :35
epoch :36
epoch :37
epoch :38
epoch :39
epoch :40
epoch :41
epoch :42
epoch :43
epoch :44
epoch :45
epoch :46
epoch :47
epoch :48
epoch :49
epoch :50
epoch :51
epoch :52
epoch :53
epoch :54
epoch :55
epoch :56
epoch :57
epoch :58
epoch :59
epoch :60
epoch :61
epoch :62
epoch :63
epoch :64
epoch :65
epoch :66
epoch :67
epoch :68
epoch :69
epoch :70
epoch :71
epoch :72
epoch :73
epoch :74
epoch :75
epoch :76
epoch :77
epoch :78
epoch :79
epoch :80
epoch :81
epoch :82
epoch :83
epoch :84
epoch :85
epoch :86
epoch :87
epoch :88
epoch :89
epoch :90
epoch :91
epoch :92
epoch :93
epoch :94
epoch :95
epoch :96
epoch :97
epoch :98
epoch :99
[519.87048